# Laboratorio 3

Si richiede un’implementazione della teoria sulle valenze di Patrick Hanks. In particolare, partendo da un corpus a scelta e uno specifico verbo (tendenzialmente non troppo frequente e/o generico ma nemmeno raro), l’idea è di costruire dei possibili cluster semantici, con relativa frequenza. Ad es. dato il verbo "to see" con valenza = 2, e usando un parser sintattico (ad es. Spacy), si possono collezionare eventuali fillers per i ruoli di subj e obj del verbo, per poi convertirli in semantic types. Un cluster frequente su "to see" potrebbe unire subj = noun.person con obj = noun.artifact. Si richiede di partire da un corpus di almeno alcune centinaia di istanze del verbo.

In [1]:
import pandas as pd
import spacy
import numpy as np
import pandas as pd
import string
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords

## Metodi di supporto

### Normalizzazione della frase

In [2]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def normalize(sentence):
    tokens = nltk.word_tokenize(sentence)
    tokens = [token for token in tokens if token not in string.punctuation] #tolgo la punteggiatura
    tokens = [token.lower() for token in tokens] # sostituisco le maiuscole con le minuscole
    tokens = [token for token in tokens if token not in stop_words] # rimuovo le stop words
    tokens = [lemmatizer.lemmatize(token) for token in tokens] # lemmatizzo
    
    return tokens

### Algoritmo Simplified Lesk

In [3]:
def simplified_lesk(word, context):
    best_sense = None
    max_overlap = 0

    for sense in wn.synsets(word):
        signature = set(normalize(sense.definition())).union(set(normalize(' '.join(sense.examples()))))
        overlap = len(context.intersection(signature))
        if overlap > max_overlap:
            max_overlap = overlap
            best_sense = sense
    
    return best_sense

## Generazione di tutte le strutture verbali per ogni verbo 

In [4]:
'''
Metodo generale che per ogni verbo tira fuori la collezione, quindi tutte le sue
valenze, andando anche a creare stesse valenze ma con pos diversi
'''
def generate_trees(corpus):
    parser = spacy.load('en_core_web_sm')
    verbs = {}

    i = 0
    for sentence in corpus['sentence']:
        sentence = sentence.replace('<s>', '').replace('</s>', '').replace('"', '').replace("'", '')[1:]
        parsified_sentence = parser(sentence)
        #spacy.displacy.render(parsified_sentence, style="dep", options={"compact": True}) # visualizza il grafo


        verb_deps = {}

        # per ogni verbo della frase prendo tutte le dipendenze di quel verbo
        for token in parsified_sentence:
            if token.head.pos_ == 'VERB':
                if token.head.lemma_ not in verb_deps:
                    verb_deps[token.head.lemma_] = set()

                if token.dep_ != 'ROOT' and token.dep_ != 'punct':
                    verb_deps[token.head.lemma_].add(token.dep_)

        # maniglia al verbo 
        current_keys = {}

        # controllo se esiste già un verbo con le stesse dipendenze altrimenti lo creo
        for verb, deps in verb_deps.items():
            current_keys[verb] = None

            # prelevo le chiavi che matchano con il verbo corrente (es. "see_1" matcha con "see")
            matching_keys = [key for key in verbs.keys() if verb in key]


            # controllo se esiste già un verbo con le stesse dipendenze
            if matching_keys:
                for key in matching_keys:
                    if deps == set(verbs[key].keys()):
                        current_keys[verb] = key

            # se non esiste un verbo con le stesse dipendenze lo creo
            if current_keys[verb] is None:
                current_keys[verb] = verb + "_" + str(i)
                verbs[current_keys[verb]] = {}
                for dep in deps:
                    verbs[current_keys[verb]][dep] = set()
                i += 1

        # A questo punto aggiungo le nuove informazioni agli slot del verbo
        for token in parsified_sentence:
            if token.dep_ != 'ROOT' and token.dep_ != 'punct':
                lemma_key = current_keys.get(token.head.lemma_)
                if lemma_key is not None:
                    verb_slot = verbs[lemma_key].get(token.dep_)
                    if verb_slot is not None:
                        synset = simplified_lesk(token.text, set(normalize(sentence)))
                        verb_slot.add((token.text, synset))
    return verbs

## Generazione delle strutture verbali con valenza 2 e slot subj e dobj

In [5]:
'''
Metodo che per ogni verbo considera solo la valenza 2 andando a fare filling degli slot nsubj e dobj
'''
def generate_trees_val_2(corpus):
    parser = spacy.load('en_core_web_sm')
    verbs = {}

    i = 0
    for sentence in corpus['sentence']:
        sentence = sentence.replace('<s>', '').replace('</s>', '').replace('"', '').replace("'", '')[1:]
        parsified_sentence = parser(sentence)
        #spacy.displacy.render(parsified_sentence, style="dep", options={"compact": True}) # visualizza il grafo


        # A questo punto aggiungo le nuove informazioni agli slot del verbo
        for token in parsified_sentence:
            if token.head.pos_ == 'VERB':
                if token.head.lemma_ not in verbs:
                    verbs[token.head.lemma_] = {
                        'nsubj': set(),
                        'dobj': set()
                    }

                if token.dep_ == 'nsubj' or token.dep_ == 'dobj':
                    synset = simplified_lesk(token.text, set(normalize(sentence)))
                    verbs[token.head.lemma_][token.dep_].add((token.text, synset))

    return verbs

## Rimozione dei verbi con slot vuoti

In [6]:
'''
Rimuovo dalla lista dei verbi quelli che non hanno istanze di nsubj o dobj
'''
def remove_verbs_with_no_nsubj_or_dobj(verbs):
    verbs_to_remove = []
    for verb, slots in verbs.items():
        if not slots['nsubj'] or not slots['dobj']:
            verbs_to_remove.append(verb)

    for verb in verbs_to_remove:
        verbs.pop(verb)

    return verbs

## Attribuzione dei semantic types

In [7]:
'''
Aggiunge i semantic types agli slot dei verbi
'''
def add_semantic_types(verbs):
    sm_slots = {}

    for verb, slots in verbs.items():
        sm_slots[verb] = {}
        
        for slot, values in slots.items():
            semantic_types = []

            for _, synset in values:
                if synset is not None:
                    semantic_types.append(synset.lexname().split('.')[1]) # ottengo il semantic type  del synset

            if semantic_types:
                semantic_type = max(set(semantic_types), key = semantic_types.count) # majority voting
                sm_slots[verb].update({slot: [semantic_type]})

    # screma i verbi che non hanno semantic type
    verbs_to_remove = []
    for verb, slots in sm_slots.items():
        if not slots:
            verbs_to_remove.append(verb)

    for verb in verbs_to_remove:
        sm_slots.pop(verb)
                
    return sm_slots

## Stampa della collezione

In [8]:
'''
per ogni verbo in verbs stampa un albero con i soggetti e gli oggetti
'''
def print_trees(verbs):
    for verb in verbs:
        print(verb)
        for dep in verbs[verb]:
            print('\t', dep)
            for word in verbs[verb][dep]:
                print('\t\t', word)
                
        print('-----------------------------')

## MAIN

In [9]:
corpus = pd.read_csv('english_wikipedia_sentence.csv')
verbs = generate_trees_val_2(corpus)
verbs = remove_verbs_with_no_nsubj_or_dobj(verbs)

In [10]:
print_trees(verbs)

begin
	 nsubj
		 ('plants', Synset('plant.n.01'))
		 ('Chaucer', None)
		 ('some', Synset('some.s.02'))
	 dobj
		 ('trend', Synset('tendency.n.04'))
-----------------------------
use
	 nsubj
		 ('who', None)
		 ('translations', Synset('transformation.n.05'))
		 ('that', None)
	 dobj
		 ('pitch', Synset('pitch.n.03'))
		 ('yupana', None)
		 ('equations', None)
		 ('hypothesis', Synset('hypothesis.n.02'))
		 ('pesticides', None)
		 ('term', Synset('term.n.02'))
-----------------------------
see
	 nsubj
		 ('year', Synset('year.n.01'))
		 ('crew', Synset('crew.v.01'))
		 ('Secretary', None)
		 ('Jung', None)
		 ('portions', Synset('part.n.01'))
		 ('goats', Synset('goat.n.01'))
		 ('Swift', Synset('fleet.s.01'))
		 ('Greeks', None)
		 ('I', Synset('one.n.01'))
		 ('Boys', Synset('male_child.n.01'))
		 ('1993', None)
		 ('he', Synset('helium.n.01'))
		 ('Gershwin', Synset('gershwin.n.01'))
		 ('we', None)
		 ('studies', Synset('study.n.05'))
		 ('distortions', Synset('distorted_shape.n.01'

In [11]:
sm_slots = add_semantic_types(verbs)
print_trees(sm_slots)

begin
	 nsubj
		 all
	 dobj
		 location
-----------------------------
use
	 nsubj
		 act
	 dobj
		 location
-----------------------------
see
	 nsubj
		 person
	 dobj
		 act
-----------------------------
ravage
	 nsubj
		 location
	 dobj
		 stative
-----------------------------
attract
	 dobj
		 stative
-----------------------------
reject
	 dobj
		 act
-----------------------------
distinguish
	 nsubj
		 attribute
	 dobj
		 cognition
-----------------------------
underlie
	 nsubj
		 artifact
	 dobj
		 event
-----------------------------
have
	 nsubj
		 all
	 dobj
		 cognition
-----------------------------
do
	 nsubj
		 group
-----------------------------
bring
	 nsubj
		 process
	 dobj
		 cognition
-----------------------------
take
	 nsubj
		 person
	 dobj
		 contact
-----------------------------
show
	 dobj
		 Tops
-----------------------------
contrast
	 dobj
		 person
-----------------------------
hurt
	 dobj
		 group
-----------------------------
send
	 nsubj
		 person
	 dobj
		 